<a href="https://colab.research.google.com/github/mamathaguntu/CMPE255/blob/master/ANN_Assignment/ANN_Algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Approximate nearest neighbor search algorithms##

Data used - Movie lens data from LightGM 
https://making.lyst.com/lightfm/docs/datasets.html



This data set contains 100,000 interactions from 1000 users on 1700 movies.


fetch_movielens() returns 


*   train 
*   test
*   item_features
*   item_feature_labels
*   item_labels

upon which the ANN algorithms are applied to find similar movies.

Algorithms used are - 


*   LSH
*   Exhaustive search
*   Product quantization
*   Trees and Graphs
*   HNSW






**Import libraries and install required packages**

In [20]:
!pip install annoy
import annoy

     |████████████████████████████████| 646 kB 29.8 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391677 sha256=4932557e1a4333afc72ca8d0a4a0c619e463c55bc7d05beabf1fe8ca68ffa6f1
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [1]:
!apt install libomp-dev
!pip install faiss

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 1s (420 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../libomp5_5.0.1-1_amd64.deb ...
Unpacking libomp5:amd64 (5.0.1-1) ...
Selecting previously unselected package libomp-dev.
Preparing to unpack .../libomp-dev_5.0.1-1_amd64.deb ...
Unpacking libomp-dev (5.0.1-

In [2]:
import faiss

In [3]:
!pip install lightfm

     |████████████████████████████████| 310 kB 31.8 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705356 sha256=faede31f3a0889e20eb2bfa4daff8e23a71fb11c62932b881cd9b810200becb7
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm


In [23]:
!pip install nmslib
import nmslib

     |████████████████████████████████| 13.5 MB 34.9 MB/s 
     |████████████████████████████████| 188 kB 52.0 MB/s 


In [4]:
import pandas as pd
from lightfm import LightFM
from lightfm.datasets import fetch_movielens

In [8]:
movie_lens_data = fetch_movielens()
movie_data = pd.DataFrame(movie_lens_data)
movie_data.head()

,train,test,item_features,item_feature_labels,item_labels
0,"(0, 0)\t5\n (0, 1)\t3\n (0, 2)\t4\n (0, 3...","(0, 19)\t4\n (0, 32)\t4\n (0, 60)\t4\n (0...","(0, 0)\t1.0\n (1, 1)\t1.0\n (2, 2)\t1.0\n ...",Toy Story (1995),Toy Story (1995)
1,"(0, 0)\t5\n (0, 1)\t3\n (0, 2)\t4\n (0, 3...","(0, 19)\t4\n (0, 32)\t4\n (0, 60)\t4\n (0...","(0, 0)\t1.0\n (1, 1)\t1.0\n (2, 2)\t1.0\n ...",GoldenEye (1995),GoldenEye (1995)
2,"(0, 0)\t5\n (0, 1)\t3\n (0, 2)\t4\n (0, 3...","(0, 19)\t4\n (0, 32)\t4\n (0, 60)\t4\n (0...","(0, 0)\t1.0\n (1, 1)\t1.0\n (2, 2)\t1.0\n ...",Four Rooms (1995),Four Rooms (1995)
3,"(0, 0)\t5\n (0, 1)\t3\n (0, 2)\t4\n (0, 3...","(0, 19)\t4\n (0, 32)\t4\n (0, 60)\t4\n (0...","(0, 0)\t1.0\n (1, 1)\t1.0\n (2, 2)\t1.0\n ...",Get Shorty (1995),Get Shorty (1995)
4,"(0, 0)\t5\n (0, 1)\t3\n (0, 2)\t4\n (0, 3...","(0, 19)\t4\n (0, 32)\t4\n (0, 60)\t4\n (0...","(0, 0)\t1.0\n (1, 1)\t1.0\n (2, 2)\t1.0\n ...",Copycat (1995),Copycat (1995)


**Data Preparation**

In [11]:
train_data = movie_lens_data['train']
test_data = movie_lens_data['test']

model = LightFM(learning_rate=0.05, loss="warp", no_components=64, item_alpha=0.001)
model.fit_partial(train_data, item_features=movie_lens_data['item_features'], epochs=20)

item_vectors = movie_lens_data['item_features'] * model.item_embeddings

fetch the movie names 

In [13]:
movie_names = movie_lens_data['item_labels']
movie_names

array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
       'Sliding Doors (1998)', 'You So Crazy (1994)',
       'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object)

###**1. Locality Sensitive Hashing**###

In [15]:
item_vectors_lsh = item_vectors.copy()
movie_names_lsh = movie_names.copy()
dimention = item_vectors_lsh.shape[1]
vectors = item_vectors_lsh.astype('float32')
labels = movie_names_lsh

quantizer = faiss.IndexFlatL2(dimention)
index = faiss.IndexIVFPQ(quantizer, dimention, 8, 2, 8)
index.train(vectors)
index.add(vectors)
        

distances, indices = index.search(item_vectors_lsh[90:91], 10) 
simlar_movies_names = [labels[i] for i in indices[0]]
simlar_movies_names = '\n'.join(simlar_movies_names)
print(f"Movies similar to {movie_names_lsh[91]} are:\n* {simlar_movies_names}")

Movies similar to True Romance (1993) are:
* Nightmare Before Christmas, The (1993)
Fantasia (1940)
Hudsucker Proxy, The (1994)
Sneakers (1992)
Snow White and the Seven Dwarfs (1937)
Batman (1989)
Aladdin (1992)
Beauty and the Beast (1991)
Dumbo (1941)
Monty Python and the Holy Grail (1974)


###**2. Exhaustive Search**###

In [17]:
item_vectors_es = item_vectors.copy()
movie_names_es = movie_names.copy()
vectors_ex = item_vectors_es.astype('float32')
labels_ex = movie_names_es
index_ex = faiss.IndexFlatL2(item_vectors_es.shape[1])
index_ex.add(vectors_ex)

distances, indices = index_ex.search(item_vectors_es[90:91], 10) 
simlar_movies_names = [labels_ex[i] for i in indices[0]]
simlar_movies_names = '\n* '.join(simlar_movies_names)
print(f"Movies similar to {movie_names_es[91]} are:\n* {simlar_movies_names}")

Movies similar to True Romance (1993) are:
* Nightmare Before Christmas, The (1993)
* Fantasia (1940)
* Beauty and the Beast (1991)
* Jumanji (1995)
* Heavy Metal (1981)
* Akira (1988)
* Indian in the Cupboard, The (1995)
* 20,000 Leagues Under the Sea (1954)
* Cook the Thief His Wife & Her Lover, The (1989)
* Lion King, The (1994)


Observe that there is a difference in the list of movies that are similar to movie 'True Romance'

###**Product Quantization**###

In [19]:
item_vectors_pq = item_vectors.copy()
movie_names_pq = movie_names.copy()
dimension_pq = item_vectors_pq.shape[1]
vectors_pq = item_vectors_pq.astype('float32')
labels_pq = movie_names_pq    
        
quantizer_pq = faiss.IndexFlatL2(dimension_pq)
index_pq = faiss.IndexIVFPQ(quantizer_pq,dimension_pq,8,2,8)
index_pq.train(vectors_pq)
index_pq.add(vectors_pq)

distances, indices = index_pq.search(item_vectors_pq[90:91], 10) 
simlar_movies_names = [labels_pq[i] for i in indices[0]]

simlar_movies_names = '\n* '.join(simlar_movies_names)
print(f"Movies similar to {movie_names_pq[91]} are:\n* {simlar_movies_names}")

Movies similar to True Romance (1993) are:
* Nightmare Before Christmas, The (1993)
* Fantasia (1940)
* Hudsucker Proxy, The (1994)
* Sneakers (1992)
* Snow White and the Seven Dwarfs (1937)
* Batman (1989)
* Aladdin (1992)
* Beauty and the Beast (1991)
* Dumbo (1941)
* Monty Python and the Holy Grail (1974)


###**Trees and Graphs**###

In [22]:
item_vectors_tg = item_vectors.copy()
movie_names_tg = movie_names.copy()
dimention_trees = item_vectors_tg.shape[1]
vectors_trees = item_vectors_tg.astype('float32')
labels_trees = movie_names_tg

    
index_trees = annoy.AnnoyIndex(dimention_trees)
for i, vec in enumerate(vectors_trees):
  index_trees.add_item(i, vec.tolist())
index_trees.build(n_trees=5)

indices = index_trees.get_nns_by_vector(item_vectors_tg[91].tolist(), 10)
simlar_movies_names = [labels_trees[i] for i in indices]
simlar_movies_names = '\n* '.join(simlar_movies_names)
        
print(f"Movies similar to {movie_names_tg[91]} are:\n* {simlar_movies_names}")

Movies similar to True Romance (1993) are:
* True Romance (1993)
* Seven (Se7en) (1995)
* GoodFellas (1990)
* Professional, The (1994)
* Swimming with Sharks (1995)
* Usual Suspects, The (1995)
* Short Cuts (1993)
* Casino (1995)
* Miller's Crossing (1990)
* Reservoir Dogs (1992)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  


###**HNSW**###

In [24]:
item_vectors_hnsw = item_vectors.copy()
movie_names_hnsw = movie_names.copy()
dimention_hnsw = item_vectors_hnsw.shape[1]
vectors_hnsw = item_vectors_hnsw.astype('float32')
labels_hnsw = movie_names_hnsw
            
index_hnsw = nmslib.init(method='hnsw', space='cosinesimil')
index_hnsw.addDataPointBatch(vectors_hnsw)
index_hnsw.createIndex({'post': 2})
        
    
indices = index_hnsw.knnQuery(item_vectors_hnsw[91], k=10)
simlar_movies_names = [labels_hnsw[i] for i in indices[0]]
simlar_movies_names = '\n* '.join(simlar_movies_names)

print(f"Movies similar to {movie_names_hnsw[91]} are:\n* {simlar_movies_names}")

Movies similar to True Romance (1993) are:
* True Romance (1993)
* Seven (Se7en) (1995)
* GoodFellas (1990)
* Professional, The (1994)
* Killing Zoe (1994)
* Swimming with Sharks (1995)
* Usual Suspects, The (1995)
* Short Cuts (1993)
* Clockers (1995)
* Casino (1995)
